# Facial Expression Recognition with TensorFlow 2.0

Mudar a versão do TensorFlow para 2.0

In [ ]:
 %tensorflow_version 2.x

Habilitar GPU


> Ir a Runtime no menu no topo --> Change runtime type --> Em Hardware accelerator, selecionar GPU.

In [ ]:
import tensorflow as tf
print(tf.__version__)
#tf.debugging.set_log_device_placement(True)

2.7.0


In [ ]:
# check for GPU, at least 1 GPU needed
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

/device:GPU:0
Num GPUs Available:  1


In [ ]:
# Check GPU information
!nvidia-smi

Sun Dec  5 15:09:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    68W / 149W |    144MiB / 11441MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fazer import dos packages necessários

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from math import sin, cos, pi

from keras.layers import Conv2D, LeakyReLU, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Sequential

In [ ]:
def plot_sample(image, keypoint, axis, title):
    image = image.reshape(96,96)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], marker='x', s=20)
    plt.title(title)

## Dataset (import, preparation, preprocessing)

> Banco de dados do desafio Kaggle, mais informações em:
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

O banco de dados possui 3 colunas: emotions | pixels | Usage

**Emotions**: são as emoções das imagens: 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

**Pixels**: é a imagem 64x64 (matriz de pixels )

**Usage**: indica se é para ser usado como trainamento ou teste






# Download dataset fer2013
! wget https://github.com/offsouza/Facial-Expression-Recognition/raw/master/fer2013.csv.zip

In [ ]:
#Unzip file .zip
! unzip https://drive.google.com/file/d/1fjrs2UzDPBuPggxv1JTpYd7mush73tpw/view?usp=sharing

unzip:  cannot find or open https://drive.google.com/file/d/1fjrs2UzDPBuPggxv1JTpYd7mush73tpw/view?usp=sharing, https://drive.google.com/file/d/1fjrs2UzDPBuPggxv1JTpYd7mush73tpw/view?usp=sharing.zip or https://drive.google.com/file/d/1fjrs2UzDPBuPggxv1JTpYd7mush73tpw/view?usp=sharing.ZIP.

No zipfiles found.


Importar banco de dados

In [ ]:
import pandas as pd
data = pd.read_csv('fer2013.csv')
data.head()


FileNotFoundError: ignored

Verificar a quantidade de dados

Existe 35887 imagens com os  seguintes labels [0 1 2 3 5 6 ]


In [ ]:
print('Count: ', data.count())
print('Emotions labels', data.emotion.unique())

Verificar a quantidade de imagens de cada emoção

In [ ]:
label_map =  { 0: "Angry", 1:'Disgust', 2:'Fear', 3:'Happy', 4:"Sad", 5:'Surprise', 6:'Neutral'}

df = data['emotion'].value_counts()
df = pd.DataFrame(df)
df = df.rename(index=label_map)
df


Nesse código não se treina todas as emoções, identificamos somente 4 emoções:

> 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

Para isso iremos ter que remover todos os dados com os Labels 0,1 e 2



In [ ]:
# Remove as expressões Angry=0, Disgust=1, fear = 2
data  = data[data['emotion'] != 0 ]
data  = data[data['emotion'] != 1 ]
data  = data[data['emotion'] != 2 ]
print('Count: ', data.count())
print('Emotions labels', data.emotion.unique() )
# aqui é realizado a substituição do labels
# Isso é feito pois, se deixar os labels de 3 a 6 irá ser gerado um vetor de 7 (0 a 6) posições na última camada, assim o modelo irá
# entender que existe 7 tipos de emoções. Assim, transformando os labels de 0 a 3, gerando vetor de 4 posições somente.
data['emotion'] = data['emotion'].replace(5, 1)
data['emotion'] = data['emotion'].replace(6, 0)
data['emotion'] = data['emotion'].replace(4, 2)

#label_map_new
# 0: Neutro, 1:Suprise, 2: Triste, 3: Happy

Converter a coluna 'pixels' de String para Texto

In [ ]:
# carregar imagens
import numpy as np

pix = data['pixels']  # Os pixels estão como texto (string) e não valor numérico, então passa-se o dataframe para um lista
pix = list(pix) # converter para lista
print("pix lista, primeiro valor: \n", pix[0:1],'\n\n') # imprimir a lista no formato formato texto
lista_pix_int = list() # criar uma lista vazia

from tqdm import tqdm   # mostrar a barra de progresso
for i in tqdm(pix):
    # aqui eu faço um split da lista para que após o numero tenha uma virgula assim mudando de ['24 32 36' ] para ['24', '32', '36']
    # no mesmo comando transforma-se de string(ex:'24') para inteiro(ex:24) assim obtendo um vetor(lista) de numero inteiros e não mais
    # de strings
    pix_int = [float(j) for j in i.split() ]
    lista_pix_int.append(pix_int) # realizar a transformação num dos vetores, adiciona-se a uma lista


array_pix = np.array(lista_pix_int) # Converter a lista com valores numéricos para um array numpy
print("pix array, primeiro valor: \n", array_pix[0:1],'\n\n') # verifica-se que a lista está em formato float
print()
print('---\nShape: ', array_pix.shape)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_n = scaler.fit_transform(array_pix)

print(df_n)

array_pix = df_n


In [ ]:
df_n[0:1]



In [ ]:
  # observando o shape (22366, 2304)
  # temos 22366 imagens que estão em um veteor de 2304 posições
  # porém sabemos que as imagens tem tamanho 48x48 = 2304

  #  Então temos que redimensionars todas as imagens para 48 x 48

print(type(array_pix))
reshape_pix = array_pix.reshape(array_pix.shape[0], 48,48,1)
print("Shape antigo:" , array_pix.shape)
print("Shape novo:" , reshape_pix.shape)


In [ ]:
# agora temos nossa imagem ou seja nossos dados de entrada no formato adequado
# Temos que dividir nossos dados em dados de treino, teste e validação
# Irei dividir dados de treino em 60% , Teste 20% e Validação 20%
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

X = reshape_pix
y = data['emotion'].values


labels = pd.DataFrame(y)
y = np_utils.to_categorical(labels)

X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.4, random_state=42,shuffle= True)

print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_teste_val: ', X_test_val.shape)
print('y_teste_val: ', y_test_val.shape)

X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, shuffle= True)
print('\n ---------------\n')
print('X_teste: ', X_test.shape)
print('y_test: ', y_test.shape)
print('X_val: ', X_val.shape)
print('y_val: ', y_val.shape)


In [ ]:
y[0]

In [ ]:
# Aqui faço o reshape novamente somente para exibir as imagens
x = X_train.reshape(X_train.shape[0], 48,48)

# plot
import matplotlib.pyplot as plt
j =0
for i in range(8, 14):
    j+=1
    plt.subplot(2,3 ,j)
    image = x[i]
    plt.imshow(image, cmap= 'gray')
    #plt.text(0,45, label_map[y_train[i]], fontsize = 20, color = 'red')
plt.show()

# Criação do Modelo CNN

A rede neural que será construida terá como base a arquitetura VGG16

In [ ]:
# importando todos os modulos

import  os
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
class Block_ConvReluBN (keras.Model):

  def __init__(self, filters, strides=1, kernel=3, padding = 'same', dp = 0.3):
    super(Block_ConvReluBN, self).__init__()

    self.conv =  keras.layers.Conv2D(filters, kernel, strides=strides, padding=padding)
    self.bn= keras.layers.BatchNormalization()
    self.relu = keras.layers.Activation('relu')
    self.drop = keras.layers.Dropout(dp)

  def call(self,x,training=None):

    x = self.conv(x )
    x = self.bn(x, training = training)
    x = self.relu(x)
    x = self.drop(x)

    return x


## Criando classe do modelo VGG16



In [ ]:
class Model_VGG16(keras.Model):

  def __init__(self,num_classes,**kwargs):
    super(Model_VGG16, self).__init__(**kwargs)

    self.num_classes = num_classes

    self.block01 = Block_ConvReluBN(64)
    self.block02 = Block_ConvReluBN(64)
    self.maxpool1 =  keras.layers.MaxPooling2D(pool_size=(2, 2))

    self.block03 = Block_ConvReluBN(128)
    self.block04 = Block_ConvReluBN(128)
    self.maxpool2 =  keras.layers.MaxPooling2D(pool_size=(2, 2))

    self.block05 = Block_ConvReluBN(256)
    self.block06 = Block_ConvReluBN(256)
    self.block07 = Block_ConvReluBN(256)
    self.maxpool3 =  keras.layers.MaxPooling2D(pool_size=(2, 2))

    self.block08 = Block_ConvReluBN(512)
    self.block09 = Block_ConvReluBN(512)
    self.block10 = Block_ConvReluBN(512)
    self.maxpool4 =  keras.layers.MaxPooling2D(pool_size=(2, 2))

    self.block11 = Block_ConvReluBN(512)
    self.block12 = Block_ConvReluBN(512)
    self.block13 = Block_ConvReluBN(512)
    self.maxpool5 =  keras.layers.MaxPooling2D(pool_size=(2, 2))

    self.flatten = keras.layers.Flatten()
    self.dense = keras.layers.Dense(512,kernel_regularizer=regularizers.l2(0.0001))
    self.bn2 = keras.layers.BatchNormalization()
    self.relu2 = keras.layers.Activation('relu')

    self.out = keras.layers.Dense(self.num_classes)

  def call(self, inputs,training=None):
    x= inputs
    x = self.block01(x)
    x = self.block02(x)
    x = self.maxpool1(x)

    x = self.block03(x)
    x = self.block04(x)
    x = self.maxpool2(x)

    x = self.block05(x)
    x = self.block06(x)
    x = self.block07(x)
    x = self.maxpool3(x)

    x = self.block08(x)
    x = self.block09(x)
    x = self.block10(x)
    x = self.maxpool4(x)

    x = self.block11(x)
    x = self.block12(x)
    x = self.block13(x)
    x = self.maxpool5(x)

    x = self.flatten(x)
    x = self.dense(x)
    x = self.relu2(x)
    x = self.bn2(x, training)
    x = self.out(x)

    return x

Build do modelo


In [ ]:
batch_size = 32
classes = 4

# build model and optimizer
model = Model_VGG16(num_classes=classes)


model.build(input_shape=(None, 48, 48,1))


print("Number of variables in the model :", len(model.trainable_variables))

model.summary()



### Treinamento usando Keras

In [ ]:
# treinamento keras
# compilando o otimizador, função de perda e metricas
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

check = ModelCheckpoint(filepath='weights_keras.hdf5', verbose=1, save_best_only=True)
#early = EarlyStopping(monitor='val_loss',patience=10)

# train
history = model.fit(X_train, y_train, batch_size=100, epochs=20, verbose=1,
         validation_data=(X_val , y_val)) # callbacks=[ check, early]


evaluate the model and compute loss and accuracy

In [ ]:
# evaluate on test set
#scores = model.evaluate(X_test, y_test, batch_size, verbose=1)
#print("Final test loss and accuracy :", scores)

sns.set_style('darkgrid')

fig, ax = plt.subplots(2, 1, figsize=(20, 10))
df = pd.DataFrame(history.history)
df[['loss', 'val_loss']].plot(ax=ax[0])
df[['accuracy', 'val_accuracy']].plot(ax=ax[1])
ax[0].set_title('Model Loss', fontsize=12)
ax[1].set_title('Model Accuracy', fontsize=12)
fig.suptitle('Model Metrics', fontsize=18);

Save the data obtained from the model

In [ ]:
model.save('model_full2', save_format="tf")


In [ ]:
!zip -r model.zip model_full/

### Treinamento Customizado com Gradient.tape())

In [ ]:
batch_size = 32
classes = 4

# build model and optimizer
model = Model_VGG16(num_classes=classes)


model.build(input_shape=(None, 48, 48,1))


print("Number of variables in the model :", len(model.trainable_variables))

model.summary()


In [ ]:

optimizer = keras.optimizers.Adam(learning_rate=0.001)
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)

acc_meter_train = tf.keras.metrics.CategoricalAccuracy(name='acc_train')

loss_metric_train = tf.keras.metrics.Mean(name='train_loss')

acc_meter_test = tf.keras.metrics.CategoricalAccuracy(name='acc_test')

loss_metric_test = tf.keras.metrics.Mean(name='test_loss')

db_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(100)
db_test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)

geral_loss = 1000
for epoch in range(50):


    for step, (x, y) in enumerate(db_train):

        with tf.GradientTape() as tape:

            logits = model(x, training=True)
            reg_loss=tf.math.add_n(model.losses)

            loss = criteon(y, logits)
            loss = loss + reg_loss


        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss_metric_train.update_state(loss)
        acc_meter_train.update_state(tf.argmax(y, axis=1),tf.argmax(logits, axis=1))

    print('------- EPOCA %s -------'%epoch)
    print('Train Acc:', acc_meter_train.result().numpy())
    print('Train Loss: ', loss_metric_train.result().numpy())


    loss_metric_train.reset_states()
    acc_meter_train.reset_states()

    for x, y in db_test:

        logits = model(x, training=False)
        val_loss = criteon(y, logits)
        pred = tf.argmax(logits, axis=1)

        y = tf.argmax(y, axis=1)

        loss_metric_test.update_state(val_loss)
        acc_meter_test.update_state(y, pred)

    print('Val acc:', acc_meter_test.result().numpy())
    print('Val loss:', loss_metric_test.result().numpy())

    val_loss = loss_metric_test.result().numpy()

    if val_loss < geral_loss:
      model.save_weights('modelo.hdf5')
      geral_loss = val_loss
      print("modelo salvo")

    loss_metric_test.reset_states()
    acc_meter_test.reset_states()

Tesntando modelo com datos de teste

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

scores = model.evaluate(X_test, y_test, verbose=1)
print("Final test loss and accuracy :", scores)

Função usada para gerar matrix de confunsão
obs: é necessario fazer algumas modificações para gerar o gráfico, por isso irei deixar comentado